In [234]:
#import libraries used into
from datetime import datetime
import pandas as pd, numpy as np, os

In [235]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
filepm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 5 P.M..XLSX"
main_path = "C:\\Users\\mruiz\\Documents\\DB OT" 
current_path = "C:\\Users\\mruiz\\Documents\\DB OT\\CurrentDocuments"
current_month = datetime.now().strftime('%B')
current_path_ = os.listdir(current_path)
main_path_ = os.listdir(main_path)

In [236]:
for item in main_path_:
    if item == filepm:
        item1 = item
        print("Los datos del día de hoy por la tarde son : ", item1)
        break
    elif item == fileam:
        item1 = item
        print("Los datos del día de hoy por la mañana son : ", item1)
        break

data1 = main_path + "\\" + item1
data1excel = pd.read_excel(data1, sheet_name="Sheet1", header=0)
print("Los datos del archivo ", item1, " fueron procesados")

Los datos del día de hoy por la mañana son :  OT 24.10.2022.XLSX
Los datos del archivo  OT 24.10.2022.XLSX  fueron procesados


In [237]:
data1excel = data1excel.assign(Material=0)

In [238]:
data1excel.rename(columns={'Inicio programado' : 'Inicio_programado', 'Plan mant.preventivo' : 'Plan_Mtto',
       'Pto.tbjo.responsable': 'Responsable', 'Status del sistema' : 'Status', 'Texto breve' : 'Descripción',
       'Denominación de objeto técnico' : 'Cliente_Equipo', 'Número de serie': 'Número_de_serie', 'Clase de orden':'Clase_de_orden',
       'Fe.liberación real':'Fecha_Liberación', 'Fecha inicio real':'Fecha_creación'}, inplace = True)

In [239]:
data1excel['Material'] = np.where((data1excel['Clase_de_orden'] == 'ZMCP') &  (data1excel['Descripción'].str.contains('Emer|Revi', case=False)), 'NO',
np.where(data1excel['Clase_de_orden'] == 'ZPM2','SI',
np.where(data1excel['Clase_de_orden'] == 'ZMCP','SI',
np.where(data1excel['Clase_de_orden'] == 'ZMCI', 'SI',
np.where((data1excel['Clase_de_orden'] == 'ZPM1') &  (data1excel['Descripción'].str.contains('CAFA|CABAT|ACUM', case=False)), 'SI','NO')))))

In [240]:
data2 =current_path + "\\Master_OT.xlsx"
data2excel = pd.read_excel(data2, sheet_name="Names", header=0)

In [241]:
data3 = current_path + "\\Politicas.XLSX"
data3excel = pd.read_excel(data3, sheet_name="POLITICAS", header=0)

In [242]:
data4 = current_path + "\\Materiales_Locales.xlsx"
data4excel = pd.read_excel(data4, sheet_name="ENTREGADO", header=0)

In [243]:
data5 = current_path + "\\Materiales_Foraneos.xlsx"
data5excel = pd.read_excel(data5, sheet_name="MATRIZ FINAL", header=1)

In [244]:
print('Las columnas de frame 1 son', data1excel.columns)
print('Las columnas de frame 2 son', data2excel.columns)
print('Las columnas de frame 3 son', data3excel.columns)
print('Las columnas de frame 4 son', data4excel.columns)
print('Las columnas de frame 5 son', data5excel.columns)

Las columnas de frame 1 son Index(['Inicio_programado', 'Orden', 'Plan_Mtto', 'Responsable', 'Status',
       'Descripción', 'Cliente', 'Cliente_Equipo', 'Número_de_serie', 'Equipo',
       'Clase_de_orden', 'Fecha_Liberación', 'Fecha_creación', 'Material'],
      dtype='object')
Las columnas de frame 2 son Index(['TecID', 'Short name', 'Estatus', 'Typetec', 'City'], dtype='object')
Las columnas de frame 3 son Index(['Equipo', 'Denominación de objeto técnico', 'Cliente actual',
       'Denominación de la ubicación técnica', 'Status del sistema',
       'Número de serie', 'TIENE POLITICA', 'TECH ASIGNADO AL EQUIPO',
       'gestora archivo anahi ', 'reportes archivo anahi ', 'Gestora',
       'Administradora Reportes', 'Tipo Equipo', 'Frecuencia',
       'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
       'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc',
       'Tiempo valido de prueba', 'Tecnicos para servicio',
       'Requerimientos adicionales', 'C

In [245]:
result_df = pd.merge(left = data1excel, right = data2excel,left_on="Responsable",right_on="TecID",how="left").drop(columns=['TecID'])

In [246]:
result_df = pd.merge(result_df,data3excel[['Equipo','TIENE POLITICA', 'TECH ASIGNADO AL EQUIPO',
       'gestora archivo anahi ', 'reportes archivo anahi ', 'Gestora',
       'Administradora Reportes', 'Tipo Equipo', 'Frecuencia',
       'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
       'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc',
       'Tiempo valido de prueba', 'Tecnicos para servicio',
       'Requerimientos adicionales', 'Correo solicitud acceso',
       'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales',
       'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS']],on="Equipo",how="left")

In [247]:
result_df = pd.merge(result_df,data4excel[['Orden', 'Inicio programado2', 'Fecha de salida',
       'Comentario', 'Material Separado', 'Columna1']], on="Orden",how= "left")

In [248]:
result_df = pd.merge(result_df,data5excel,on='Orden',how='left')

In [249]:
print(result_df.loc[0])

Inicio_programado    2024-07-26 00:00:00
Orden                            4096274
Plan_Mtto                         8396.0
Responsable                     TECSER24
Status                         BLOQUEADA
                            ...         
Unnamed: 25                          NaN
Unnamed: 26                          NaN
Unnamed: 27                          NaN
Unnamed: 28                          NaN
Unnamed: 29                          NaN
Name: 0, Length: 79, dtype: object


In [250]:
result_df.assign(Pendiente_Recoger=0)

,Inicio_programado,Orden,Plan_Mtto,Responsable,Status,Descripción,Cliente,Cliente_Equipo,Número_de_serie,Equipo,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Pendiente_Recoger
0,2024-07-26,4096274,8396.0,TECSER24,BLOQUEADA,SEGUIDORES DE LA CRUZ 10KW,303371.0,SEGUIDORES DE LA CRUZ 10KW,NaN,12923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2023-12-12,4093006,626.0,TECSER17,ABIERTA,HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA,300105.0,HD MIGUEL ALEMAN 8789 PTA 350KW,43203341,10373.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2023-10-12,4093001,439.0,TECSER6,ABIERTA,HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL,300105.0,HD PIEDRAS NEGRAS 8638 PTA 350KW,2012658048,10270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2023-09-13,4093222,1522.0,TECSER2,ABIERTA,HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR,300105.0,HD INTERLOMAS 8762 PTA 250KW,14808,10935.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2023-09-11,4093049,653.0,TECSER6,ABIERTA,HD SAN PEDRO 8652 PTA 350KW ACUMULADOR,300105.0,HD SAN PEDRO 8652 PTA 350KW,UI4182W,10388.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39898,2021-01-02,4056441,4393.0,TECSER9,ABIERTA,CASA CHAPA PLANTA GUADALUPE CAFAM,300065.0,CASA CHAPA GUADALUPE PTA 40KW,46236915,10375.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
39899,2021-01-01,4066140,NaN,TECSER23,NOTIFICADA,AURRERA RUIZ CORTINEZ 3713 EMERGENCIA,300107.0,AURRERA RUIZ CORTINEZ 3713 PTA 600KW,UI40IIV,10428.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
39900,2021-01-01,4066145,NaN,TECSER6,NOTIFICADA,EDIFICIO CHROMA EMERGENCIA,301784.0,EDIFICIO CHROMA PTA 400 KW,16501002985,11530.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
39901,2021-01-01,4057764,5581.0,TECSER29,BLOQUEADA,DEL SOL GOMEZ PALACIOS 137 CAFAM,300099.0,DEL SOL GOMEZ PALACIOS 137 PTA 400KW,43203929,11390.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [251]:
result_df['Pendiente_Recoger'] = np.where(result_df['Inicio programado2'].isnull(),result_df['ESTATUS COMPRA'],result_df['Inicio programado2'])

In [252]:
print(result_df.columns)

Index(['Inicio_programado', 'Orden', 'Plan_Mtto', 'Responsable', 'Status',
       'Descripción', 'Cliente', 'Cliente_Equipo', 'Número_de_serie', 'Equipo',
       'Clase_de_orden', 'Fecha_Liberación', 'Fecha_creación', 'Material',
       'Short name', 'Estatus', 'Typetec', 'City', 'TIENE POLITICA',
       'TECH ASIGNADO AL EQUIPO', 'gestora archivo anahi ',
       'reportes archivo anahi ', 'Gestora', 'Administradora Reportes',
       'Tipo Equipo', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo',
       'Dia o fecha del Serv.', 'Horario', 'Curso', 'DC-3',
       'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba',
       'Tecnicos para servicio', 'Requerimientos adicionales',
       'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS',
       'REPORTE ESPECIAL', 'Incluye materiales', 'OC', 'FRECUENCIA DE OC',
       'PREVENTIVOS', 'CORRECTIVOS', 'Inicio programado2', 'Fecha de salida',
       'Comentario', 'Material Separado', 'Columna1', 'ESTATUS OT',
       'Unnamed:

In [253]:
columnasdeseadas = ['Inicio_programado','Fecha_Liberación','Orden','Descripción', 'Short name','Pendiente_Recoger', 'Fecha de salida', 'Comentario']
result_df2 = result_df[columnasdeseadas]

In [258]:
os.remove(current_path + "\\Main.xlsx")
path = current_path + "\\Main.xlsx"
result_df.to_excel(path, index=False,header=True)
with pd.ExcelWriter(path) as writer:
    result_df.to_excel(writer,index=False,sheet_name="DB")
    result_df2.to_excel(writer,index=False ,sheet_name="Material")

PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\mruiz\\Documents\\DB OT\\CurrentDocuments\\Main.xlsx'